In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import torchtext.experimental
import torchtext.experimental.vectors

import collections
import time
import random

In [25]:
from classifier import *
from helpers import ep_time, predict

In [4]:
seed = 1234
max_len = 500
max_size = 25000

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
raw_train, raw_test = torchtext.experimental.datasets.raw.IMDB()
raw_train, raw_valid = get_train_valid_split(raw_train)

In [6]:
tokenizer = Tokenizer(max_len=max_len)

In [7]:
vocab = gen_vocab(raw_train, tokenizer, max_size=max_size)

In [8]:
train_data = process_raw(raw_train,tokenizer, vocab)
test_data = process_raw(raw_test,tokenizer, vocab)
valid_data = process_raw(raw_valid,tokenizer, vocab)

In [19]:
batch_size = 256
pad_token = '<pad>'
unk_token = '<unk>'
pad_idx = vocab[pad_token]
input_dim = len(vocab)
emb_dim = 100
hid_dim = 256
output_dim = 2
n_layers = 2
dropout = 0.5
n_epochs = 10
best_valid_loss = float('inf')

In [10]:
collator = Collator(pad_idx)

In [11]:
train_iterator = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True, collate_fn=collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size, shuffle=False, collate_fn=collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False, collate_fn=collator.collate)

In [12]:
model = BiLSTM(input_dim, emb_dim, hid_dim, output_dim, n_layers, dropout, pad_idx)

glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)
# for n,p in model.named_parameters():
#     print(f'name:{n}\nshape:{p.shape}\n')
model.apply(init_params)

BiLSTM(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [13]:
pretrained_embedding, unk_tokens = get_pretrained_embedding(model.embedding, glove, vocab, unk_token)

In [14]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[-0.0398,  0.0357, -0.0046,  ..., -0.0485, -0.0088,  0.0329],
        [-0.0330,  0.0428,  0.0304,  ...,  0.0236,  0.0487,  0.0101],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [15]:
model.embedding.weight.data[pad_idx] = torch.zeros(emb_dim)

In [16]:
optimizer = optim.Adam(model.parameters())

In [17]:
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [23]:
for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = ep_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bilstm.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 50s
	Train Loss: 0.743 | Train Acc: 53.14%
	 Val. Loss: 0.681 |  Val. Acc: 55.45%
Epoch: 02 | Epoch Time: 0m 51s
	Train Loss: 0.671 | Train Acc: 59.47%
	 Val. Loss: 0.576 |  Val. Acc: 70.10%
Epoch: 03 | Epoch Time: 0m 51s
	Train Loss: 0.540 | Train Acc: 72.85%
	 Val. Loss: 0.421 |  Val. Acc: 81.69%
Epoch: 04 | Epoch Time: 0m 52s
	Train Loss: 0.379 | Train Acc: 83.66%
	 Val. Loss: 0.351 |  Val. Acc: 86.65%
Epoch: 05 | Epoch Time: 0m 52s
	Train Loss: 0.299 | Train Acc: 87.80%
	 Val. Loss: 0.292 |  Val. Acc: 88.16%
Epoch: 06 | Epoch Time: 0m 55s
	Train Loss: 0.254 | Train Acc: 89.90%
	 Val. Loss: 0.295 |  Val. Acc: 89.37%
Epoch: 07 | Epoch Time: 0m 53s
	Train Loss: 0.228 | Train Acc: 91.18%
	 Val. Loss: 0.280 |  Val. Acc: 89.82%
Epoch: 08 | Epoch Time: 0m 54s
	Train Loss: 0.199 | Train Acc: 92.35%
	 Val. Loss: 0.285 |  Val. Acc: 90.19%
Epoch: 09 | Epoch Time: 0m 55s
	Train Loss: 0.169 | Train Acc: 93.75%
	 Val. Loss: 0.322 |  Val. Acc: 88.45%
Epoch: 10 | Epoch T

In [24]:
model.load_state_dict(torch.load('bilstm.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.299 | Test Acc: 88.46%


In [29]:
sent = 'I fucking hate you'
predict(tokenizer, vocab, model, device, sent)

0.3840201199054718